<a href="https://colab.research.google.com/github/yohoobot/works/blob/main/qwen2_5_replicateMusicGen_yes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.environ["REPLICATE_API_TOKEN"] = " "  # 替换成你的 API Key

!curl -s -H "Authorization: Bearer $REPLICATE_API_TOKEN" https://api.replicate.com/v1/account



{"type":"user","username":"yohoobot","name":"","avatar_url":"https://github.com/yohoobot.png","github_url":"https://github.com/yohoobot"}

In [ ]:
!pip install replicate

In [5]:
import json
import requests
import random
import replicate
from IPython.display import Audio

# ✅ 你的 API Key（请替换）
QWEN_API_KEY = "sk- "  # 替换为你的 Qwen2.5 API Key
REPLICATE_API_KEY=" "

# ✅ 配置 API 端点
QWEN_API_URL = "https://dashscope.aliyuncs.com/api/v1/services/aigc/text-generation/generation"


# ✅ 设置 Replicate API

replicate.client = replicate.Client(api_token=REPLICATE_API_KEY)

# 读取 JSON 数据集
with open("using.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# 选取 Few-shot 示例
few_shot_examples = random.sample(dataset, 5)
few_shot_prompt = "\n".join(
    [f"Scene: {pair['scene']}\nMusic: {pair['music_desc']}\n" for pair in few_shot_examples]
)

def generate_music_description(scene_desc):
    """使用 Qwen2.5-7B-Instruct 生成背景音乐描述"""
    messages = [
        {"role": "system", "content": "You are an expert in restaurant music selection. Based on the given restaurant scene description, generate a suitable background music description."},
        {"role": "user", "content": f"Here are some examples:\n{few_shot_prompt}\n\nNow, based on the following scene, generate a matching music description.\n\nScene: {scene_desc}\nMusic:"}
    ]

    headers = {
        "Authorization": f"Bearer {QWEN_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "qwen2.5-7b-instruct",
        "input": {"messages": messages},
        "parameters": {
            "temperature": 0.7,
            "max_tokens": 100
        }
    }

    response = requests.post(QWEN_API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        return result.get("output", {}).get("text", "No output received.")
    else:
        return f"Error: {response.text}"

def generate_music_from_text(music_desc, duration=10):
    """使用 Replicate 的 MusicGen 生成音频"""
    print(f"Generating music using MusicGen: {music_desc}")

    input_params = {
        "prompt": music_desc,
        "model_version": "stereo-large",
        "output_format": "mp3",
        "normalization_strategy": "peak"
    }

    output = replicate.run(
        "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
        input=input_params
    )

    if isinstance(output, str):
        audio_url = output
    elif isinstance(output, list) and len(output) > 0:
        audio_url = output[0]
    else:
        return "Error: No audio generated"

    # 下载音频文件
    !wget -O generated_music.mp3 {audio_url}

    return "generated_music.mp3"

# 示例测试
test_scene = "现代餐厅，午餐时段，人流量中等"
music_desc = generate_music_description(test_scene)
print("Generated Music Description:", music_desc)

if "Error" not in music_desc:
    audio_file = generate_music_from_text(music_desc)

    if "Error" not in audio_file:
        # 播放音频
        display(Audio(audio_file))

Generated Music Description: Music: 轻快流畅的现代爵士乐，搭配电子键盘和贝斯的和谐演奏，营造出轻松愉快且不失时尚感的用餐氛围。
Generating music using MusicGen: Music: 轻快流畅的现代爵士乐，搭配电子键盘和贝斯的和谐演奏，营造出轻松愉快且不失时尚感的用餐氛围。


ReplicateError: ReplicateError Details:
title: Billing required
status: 402
detail: You need to set up billing to run this model. Go to https://replicate.com/account/billing#billing to set it up. If you have recently set up billing, please wait a few minutes before trying again.